In this mission, we'll be working with a csv file containing index prices. Each row in the file contains a daily record of the price of the S&P500 Index from 1950 to 2015. The dataset is stored in sphist.csv.

The columns of the dataset are:
Date -- The date of the record.
Open -- The opening price of the day (when trading starts).
High -- The highest trade price during the day.
Low -- The lowest trade price during the day.
Close -- The closing price for the day (when trading is finished).
Volume -- The number of shares traded.
Adj Close -- The daily closing price, adjusted retroactively to include any corporate actions

We'll be using this dataset to develop a predictive model. You'll train the model with data from 1950-2012, and try to make predictions from 2013-2015.

In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.linear_model import LinearRegression

data=pd.read_csv(r"C:\Users\malay\Downloads\sphist.csv")
data.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2015-12-07,2090.419922,2090.419922,2066.780029,2077.070068,4.043820e+09,2077.070068
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117
3,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010
4,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883


In [75]:
data["Date"]=pd.to_datetime(data.Date, format="%Y-%m-%d")
data=data.sort_values(by=["Date"], ascending=True)
#data.head(10)

Datasets taken from the stock market need to be handled differently than datasets from other sectors when it comes time to make predictions. In a normal machine learning exercise, we treat each row as independent. Stock market data is sequential, and each observation comes a day after the previous observation. Thus, the observations are not all independent, and we can't treat them as such. This means we have to be extra careful to not inject "future" knowledge into past rows when doing training and prediction. Injecting future knowledge will make our model look good when we're training and testing it, but will make it fail in the real world.

The time series nature of the data means that can generate indicators to make our model more accurate. For instance, we can create a new column that contains the average price of the last 10 trades for each row. This will incorporate information from multiple prior rows into one, and will make predictions much more accurate. When we do this, we have to be careful not to use the current row in the values you average. We want to teach the model how to predict the current price from historical prices. If we include the current price in the prices we average, it will be equivalent to handing the answers to the model upfront, and will make it impossible to use in the "real world", where we don't know the price upfront.

In [76]:
dfdata_two=data.copy()
dfdata_two["data_mean_5day"] = dfdata_two["Close"].rolling(window=5).mean().shift(1)
dfdata_two["data_mean_30day"] = dfdata_two["Close"].rolling(window=30).mean().shift(1)
dfdata_two["data_mean_365day"] = dfdata_two["Close"].rolling(window=365).mean().shift(1)
dfdata_two["data_mean_ratio"] = dfdata_two["data_mean_5day"]/dfdata_two["data_mean_365day"]

dfdata_two["data_std_5day"] = dfdata_two["Close"].rolling(window=5).std().shift(1)
dfdata_two["data_std_365day"] = dfdata_two["Close"].rolling(window=365).std().shift(1)
dfdata_two["data_std_ratio"] = dfdata_two["data_std_5day"]/dfdata_two["data_std_365day"]

dfdata_two.head(10)

,Date,Open,High,Low,Close,Volume,Adj Close,data_mean_5day,data_mean_30day,data_mean_365day,data_mean_ratio,data_std_5day,data_std_365day,data_std_ratio
16589,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000.0,16.660000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16588,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000.0,16.850000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16587,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000.0,16.930000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16586,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000.0,16.980000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16585,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000.0,17.080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16584,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000.0,17.030001,16.900,NaN,NaN,NaN,0.157956,NaN,NaN
16583,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000.0,17.090000,16.974,NaN,NaN,NaN,0.089051,NaN,NaN
16582,1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000.0,16.760000,17.022,NaN,NaN,NaN,0.067602,NaN,NaN
16581,1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000.0,16.670000,16.988,NaN,NaN,NaN,0.134796,NaN,NaN
16580,1950-01-16,16.719999,16.719999,16.719999,16.719999,1460000.0,16.719999,16.926,NaN,NaN,NaN,0.196545,NaN,NaN


Since the mean calculation is for 365 days also, and the data range is huge (from 1950-03-01) we will remove the oldest 365 rows so there are no NaN values in mean

In [77]:
df_new=dfdata_two[dfdata_two["Date"]>datetime(year=1951, month=6, day=18)]

We will generate two new dataframes to use in making our algorithm. train should contain any rows in the data with a date less than 2013-01-01. test should contain any rows with a date greater than or equal to 2013-01-01.

In [78]:
df_train=df_new[df_new["Date"]<datetime(year=2013, month=1, day=1)]
df_test=df_new[df_new["Date"]>=datetime(year=2013, month=1, day=1)]
# print(df_train.head())
# print(df_test.head())

In [79]:
features = ['data_mean_5day', 'data_mean_365day', 'data_mean_ratio', 'data_std_5day', 'data_std_365day', 'data_std_ratio']
lr=LinearRegression()
X_train=df_train[features]
X_test=df_test[features]
y_train=df_train["Close"]
y_test=df_test["Close"]

lr.fit(X_train, y_train)
predictions=lr.predict(X_test)

Compute the error between the predictions and the Close column of test using MAE

In [80]:
MAE = sum(abs(predictions - y_test))/len(predictions)
print(MAE)
print(lr.score(X_train, y_train))

16.131123821169886
0.9995187793989123
